In [2]:
import VariationalBayes
from VariationalBayes import VectorParam, ScalarParam, PosDefMatrixParam
from autograd import grad, hessian, jacobian
import math
import autograd.numpy as np
import autograd.numpy.random as npr
import copy

In [5]:
def taylor_sine(x):  # Taylor approximation to sine function
    ans = currterm = x
    i = 0
    while np.abs(currterm) > 0.001:
        currterm = -currterm * x**2 / ((2 * i + 3) * (2 * i + 2))
        ans = ans + currterm
        i += 1
    return ans

grad_sine = grad(taylor_sine)
print "Gradient of sin(pi) is", grad_sine(np.pi)  

 Gradient of sin(pi) is -0.999899529704


In [16]:
# Check Hessian of a quadratic function.
D = 5
H = npr.randn(D, D)
def fun(x, y):
    return np.dot(np.dot(x, H),x) + 10 * np.dot(y, np.matmul(H, y))

hess_x = hessian(fun, 0)
hess_y = hessian(fun, 1)

x = npr.randn(D)
print hess_x(x, x)
print 2 * H
hess_x(x, x) - (H + H.T)

[[-2.20461278  4.0156665   2.01176336  0.02641392  2.00233076]
 [ 4.0156665  -3.09432171 -0.66416112  0.29533287 -0.00873852]
 [ 2.01176336 -0.66416112  0.02358476 -1.09047981 -0.49370142]
 [ 0.02641392  0.29533287 -1.09047981 -4.05210016  0.75595788]
 [ 2.00233076 -0.00873852 -0.49370142  0.75595788  0.56740476]]
[[-2.20461278  4.20265742  1.86859186  0.49389384  1.22554712]
 [ 3.82867557 -3.09432171 -0.91930598 -2.03390492 -2.42084736]
 [ 2.15493486 -0.40901625  0.02358476 -2.35021807 -1.35420424]
 [-0.441066    2.62457066  0.16925846 -4.05210016  1.37779237]
 [ 2.77911439  2.40337031  0.3668014   0.13412338  0.56740476]]


array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [4]:
fun = lambda x: np.array([x, x**2, x**3])
val = npr.randn()
fun_jac = jacobian(fun)
fun_jac(val)  - np.array([1., 2*val, 3*val**2])

array([ 0.,  0.,  0.])

In [5]:
x = float("inf")
x == float("inf")
vec = np.array([1, 2, 3])
vec.size
vec_free = np.empty_like(vec)
vec_free[0] = 5
print vec_free
print 2 - np.exp(vec)
all(vec < 100)
False and all(False)


[5 0 0]
[ -0.71828183  -5.3890561  -18.08553692]


False

In [6]:
print constrain(unconstrain_vector(np.array([1., 2., 3.]), -1.0, 4.0), -1.0, 4.0)
print constrain(unconstrain_vector(np.array([1., 2., 3.]), -1.0, float("inf")), -1.0, float("inf"))
print constrain(unconstrain_vector(np.array([1., 2., 3.]), -1 * float("inf"), 4.0), -1 * float("inf"), 4.0)
print constrain(unconstrain_vector(np.array([1., 2., 3.]), -1 * float("inf"), float("inf")), -1 * float("inf"), float("inf"))



NameError: name 'constrain' is not defined

In [ ]:
range(1, 10)

In [ ]:

class VectorParam(object):
    def __init__(self, name, size, lb=-float("inf"), ub=float("inf")):
        self.name = name
        self.__size = size
        self.__free_size = size
        self.__val = np.empty(size)
        if lb >= ub: raise ValueError('Upper bound must strictly exceed lower bound')
        self.__lb = lb
        self.__ub = ub
    def __str__(self):
        return self.name + ': ' + str(self.__val)
    def names(self):
        return [ self.name + '_' + str(k) for k in range(self.size()) ]
    def set(self, val):
        if val.size != self.size(): raise ValueError('Wrong size for vector ' + self.name)
        self.__val = val
    def get(self):
        return self.__val
    def set_free(self, free_val):
        if free_val.size != self.size(): raise ValueError('Wrong size for vector ' + self.name)
        self.set(constrain(free_val, self.__lb, self.__ub))
    def get_free(self):
        return unconstrain_vector(self.__val, self.__lb, self.__ub)
    def size(self):
        return self.__size
    def free_size(self):
        return self.__free_size

    
class ScalarParam(object):
    def __init__(self, name, lb=-float("inf"), ub=float("inf")):
        self.name = name
        if lb >= ub: raise ValueError('Upper bound must strictly exceed lower bound')
        self.__val = 0.5 * (ub + lb)
        self.__lb = lb
        self.__ub = ub
    def __str__(self):
        return self.name + ': ' + str(self.__val)
    def names(self):
        return [ self.name ]
    def set(self, val):
        self.__val = val
    def get(self):
        return self.__val
    def set_free(self, free_val):
        self.set(constrain(free_val, self.__lb, self.__ub))
    def get_free(self):
        return unconstrain_scalar(self.__val, self.__lb, self.__ub)
    def size(self):
        return 1
    def free_size(self):
        return 1


mu = VectorParam("mu", 3, lb=0, ub=10)
mu.set(np.array([1., 2., 3.]))
mu.get()
foo = mu.get_free()
mu.set_free(foo)
mu.get()

tau = ScalarParam('tau', lb=0, ub=float("inf"))
tau.set(5)
# print dir(tau)
tau.get_free()
print tau
print mu

In [ ]:

a = np.matrix([[1, 0.1], [0.1, 1]])
print a
a_chol_orig = np.linalg.cholesky(a)
print a_chol_orig
print a_chol_orig * a_chol_orig.T - a


nrow, ncol = np.shape(a)

In [ ]:
import math

# Uses 0-indexing.
def sym_index(row, col):
    if row <= col:
        return col * (col + 1) / 2 + row
    else:
        return row * (row + 1) / 2 + col
        

def vectorize_matrix(mat, ld=True):
    nrow, ncol = np.shape(mat)
    if nrow != ncol: raise ValueError('mat must be square')
    vec_size = nrow * (nrow + 1) / 2
    vec = np.empty(vec_size)
    for col in range(ncol):
        for row in range(col + 1):
            if ld:
                vec[sym_index(row, col)] = mat[col, row]                
            else:
                vec[sym_index(row, col)] = mat[row, col]
    return vec

def unvectorize_matrix(vec, ld_only=False):
    mat_size = int(0.5 * (math.sqrt(1 + 8 * vec.size) - 1))
    if mat_size * (mat_size + 1) / 2 != vec.size: raise ValueError('Vector is an impossible size')
    mat = np.matrix(np.zeros([ mat_size, mat_size ]))
    for row in range(mat_size):
        for col in range(row + 1):
            mat[row, col] = vec[sym_index(row, col)]
            if (not ld_only) and row != col:
                mat[col, row] = mat[row, col]
    return mat


def pack_posdef_matrix(mat):
    return vectorize_matrix(np.linalg.cholesky(mat), ld=True)

def unpack_posdef_matrix(free_vec):
    mat_chol = unvectorize_matrix(free_vec, ld_only=True)
    return mat_chol * mat_chol.T

a = np.matrix(np.random.rand(4, 4))
sigma_val = a * a.T + np.eye(4)

print sigma_val
# vec = vectorize_matrix(sigma_val)
# print vec
# print unvectorize_matrix(vec)

print '-------\n'
print np.linalg.cholesky(sigma_val)
print vectorize_matrix(np.linalg.cholesky(sigma_val), ld=True)
print unvectorize_matrix(vectorize_matrix(np.linalg.cholesky(sigma_val), ld=True), ld_only=True)

unpack_posdef_matrix(pack_posdef_matrix(sigma_val)) - sigma_val


In [ ]:
class PosDefMatrixParam(object):
    def __init__(self, name, size):
        self.name = name
        self.__size = size
        self.__vec_size = size * (size + 1) / 2
        self.__val = np.matrix(np.zeros([size, size]))
    def __str__(self):
        return self.name + ': ' + str(self.__val)
    def names(self):
        return [ self.name ]
    def set(self, val):
        nrow, ncol = np.shape(val)
        if nrow != self.__size or ncol != self.__size: raise ValueError('Matrix is a different size')
        self.__val = val
    def get(self):
        return self.__val
    def set_free(self, free_val):
        if free_val.size != self.__vec_size: raise ValueError('Free value is the wrong length')
        self.set(unpack_posdef_matrix(free_val))
    def get_free(self):
        return pack_posdef_matrix(self.__val)
    def size(self):
        return self.__size
    def free_size(self):
        return self.__vec_size


    
sigma = PosDefMatrixParam('sigma', 4)
sigma.set(sigma_val)
sigma_0 = PosDefMatrixParam('sigma0', 4)
sigma_0.set_free(sigma.get_free())
print sigma_0.get() - sigma.get()

In [ ]:
class ModelParamsList(object):
    def __init__(self):
        self.param_list = []
        self.__size = 0
        self.__free_size = 0
    def __str__(self):
        return 'ModelParamsList:\n' + '\n'.join([ '\t' + str(param) for param in self.param_list ])
    def push_param(self, param):
        self.param_list.append(param)
        self.__size = self.__size + param.size()
        self.__free_size = self.__free_size + param.size()
    def set(self, vec):
        if vec.size != self.__size: raise ValueError("Wrong size.")
        offset = 0
        for param in self.param_list:
            param.set(vec[offset:(offset + param.size())])
            offset = offset + param.size()
    def get(self):
        vec = np.empty(self.size())
        offset = 0
        for param in self.param_list:
            vec[offset:(offset + param.size())] = param.get()
            offset = offset + param.size()
        return vec
    def set_free(self, vec):
        if vec.size != self.__free_size: raise ValueError("Wrong size.")
        offset = 0
        for param in self.param_list:
            param.set_free(vec[offset:(offset + param.free_size())])
            offset = offset + param.free_size()
    def get_free(self):
        vec = np.empty(self.free_size())
        offset = 0
        for param in self.param_list:
            vec[offset:(offset + param.free_size())] = param.get_free()
            offset = offset + param.free_size()
        return vec
    def names(self):
        return np.concatenate([ param.names() for param in self.param_list])
    def size(self):
        return self.__size
    def free_size(self):
        return self.__free_size

par = ModelParamsList()
par.push_param(VectorParam("mu", 3))        
par.push_param(VectorParam("sigma", 3, lb=0))        
par.push_param(ScalarParam("tau", lb=0, ub=1))        


par.param_list[0].set(np.array([-10, 20, -30]))
par.param_list[1].set(np.array([1, 2, 3]))
par.param_list[2].set(0.5)

print par.param_list[2].get()


print par.names()
print par.get()
print par.get_free()

print par


In [ ]:
class ModelParamsDict(object):
    def __init__(self):
        self.param_dict = {}
        self.__size = 0
        self.__free_size = 0
    def __str__(self):
        return 'ModelParamsList:\n' + '\n'.join([ '\t' + str(param) for param in self.param_dict.values() ])
    def __getitem__(self, key):
        return self.param_dict[key]
    def push_param(self, param):
        self.param_dict[param.name] = param
        self.__size = self.__size + param.size()
        self.__free_size = self.__free_size + param.size()
    def set(self, vec):
        if vec.size != self.__size: raise ValueError("Wrong size.")
        offset = 0
        for param in self.param_dict.values():
            param.set(vec[offset:(offset + param.size())])
            offset = offset + param.size()
    def get(self):
        vec = np.empty(self.size())
        offset = 0
        for param in self.param_dict.values():
            vec[offset:(offset + param.size())] = param.get()
            offset = offset + param.size()
        return vec
    def set_free(self, vec):
        if vec.size != self.__free_size: raise ValueError("Wrong size.")
        offset = 0
        for param in self.param_dict.values():
            param.set_free(vec[offset:(offset + param.free_size())])
            offset = offset + param.free_size()
    def get_free(self):
        vec = np.empty(self.free_size())
        offset = 0
        for param in self.param_dict.values():
            vec[offset:(offset + param.free_size())] = param.get_free()
            offset = offset + param.free_size()
        return vec
    def names(self):
        return np.concatenate([ param.names() for param in self.param_dict.values()])
    def size(self):
        return self.__size
    def free_size(self):
        return self.__free_size

par = ModelParamsDict()
par.push_param(VectorParam("mu", 3))        
par.push_param(VectorParam("sigma", 3, lb=0))        
par.push_param(ScalarParam("tau", lb=0, ub=1))        


par.param_dict['mu'].set(np.array([-10, 20, -30]))
par.param_dict['sigma'].set(np.array([1, 2, 3]))
par.param_dict['tau'].set(0.5)

print par['tau'].get()


print par.names()
print par.get()
print par.get_free()

print par
print par['mu']

In [ ]:
np.concatenate((np.array([1,2]), np.array([3,4]), [5]), axis=0)
foo = [1, 2, 3, 4]
offset = 1
print foo[offset:(offset + 1)]
print foo[0:3]

foo = {}
foo['yues'] = 5
foo['blue'] = ScalarParam("mu")
print foo

for k, v in foo.iteritems():
    print k
    print v
    print '-------'
    
print foo.values()
    

In [46]:
from autograd.core import primitive

def exp_matrix_diagonal(mat):
    assert mat.shape[0] == mat.shape[1]
    mat_exp_diag = np.make_diagonal(np.exp(np.diag(mat)), offset=0, axis1=-1, axis2=-2)
    mat_diag = np.make_diagonal(np.diag(mat), offset=0, axis1=-1, axis2=-2)
    return mat_exp_diag + mat - mat_diag


def log_matrix_diagonal(mat):
    assert mat.shape[0] == mat.shape[1]
    mat_log_diag = np.make_diagonal(np.log(np.diag(mat)), offset=0, axis1=-1, axis2=-2)
    mat_diag = np.make_diagonal(np.diag(mat), offset=0, axis1=-1, axis2=-2)
    return mat_log_diag + mat - mat_diag

mat = 0.01 * np.array(np.arange(9),  dtype=float).reshape(3, 3) + 2

print exp_matrix_diagonal(mat)
ExpMatrixDiagonalGrad = jacobian(exp_matrix_diagonal)
ExpMatrixDiagonalHess = hessian(exp_matrix_diagonal)
print ExpMatrixDiagonalGrad(mat)
print ExpMatrixDiagonalHess(mat)

# print log_matrix_diagonal(mat)
# LogMatrixDiagonalGrad = jacobian(log_matrix_diagonal)
# LogMatrixDiagonalHess = hessian(log_matrix_diagonal)
# print LogMatrixDiagonalGrad(mat)
# print LogMatrixDiagonalHess(mat)

[[ 7.3890561   2.01        2.02      ]
 [ 2.03        7.6906092   2.05      ]
 [ 2.06        2.07        8.00446891]]
[[[[ 7.3890561   0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          1.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          1.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]]


 [[[ 0.          0.          0.        ]
   [ 1.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          7.6906092   0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          1.        ]
   [ 0.          0.          0.        ]]]


 [[[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 1.          0.          0.        ]]

  [